In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect
import math
import os

In [5]:
def getURL(page):
    """
    :param page: html of web page (here: Python home page) 
    :return: urls in that page 
    """
    start_link = page.find("a href")
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1: end_quote]
    return url, end_quote

def PageScraper(url):
    l=[]
    response = requests.get(url)
    page = str(BeautifulSoup(response.content))
    while url!= '#comment_form':
        url, n = getURL(page)
        page = page[n:]
        if url and url[:5]=='/book':
            l.append(url)
    return l

#Scraps the first 100 pages
outfile = open("Pages101-200.txt", "a")
for k in range(101,201):
    url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever?page='+str(k)
    L = PageScraper(url)
    for k in range(len(L)):
        outfile.write('https://www.goodreads.com/'+L[k])
        outfile.write("\n")
outfile.close()

In [10]:
#Creating folders fo each page
path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3"
os.chdir(path)
for i in range(101):
    if 0<i<10:
        os.makedirs("page_10"+str(i))
    if 9<i<100:
        os.makedirs("page_1"+str(i))
    if i==100:
        os.makedirs("page_200")

In [30]:
# First, I make sure i am in the directory that has the text file with the URLS
path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3"
os.chdir(path)

i = 9901
page = 100

with open("Pages101-200.txt", "r") as urls:
    # I previously created the 100 directories, from page 101 to page 200
    # I change the working directory for which i want the HTMLs 
    path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_200"
    os.chdir(path)

    for url in urls.readlines():

        html = urlopen(url)

        soup = BeautifulSoup(html.read(), 'html.parser')
        
        # The logic here is not the best but it works- 
        # Basically, once every 100 urls treated I change the directory
        
        if i <= page*100:
                # We stay in the current directory
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

            i = i + 1

        if i == (page*100)+1 and 0<page<9:
            page = page + 1
            path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_10"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))
            
            
        if i == (page*100)+1 and 8<page<99:
            page = page + 1
            path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_1"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))
            
            
        if i == (page*100)+1 and page==99:
            page = page + 1
            path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_"+str(page*2)
            os.chdir(path)
    
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))
            
            

ValueError: unknown url type: ''

In [4]:
# SECTION 1

# All functions to retrieve the informations

def get_title(soup):
    try:
        title = soup.find(id="bookTitle").contents[0].strip()
        return title
    except AttributeError:
        return ""

def get_series(soup):
    try:
        serie = soup.find(id="bookSeries").get_text().strip().replace("(","").replace(")","")
        regex = "#.*"
        serie = re.sub(regex, "",serie)
        return serie
    except AttributeError:
        return ""

def get_author(soup):
    try:
        author = soup.find_all(class_="authorName__container")
        author_list = " ".join([x.get_text().strip() for x in author])
        return author_list
    except AttributeError:
        return ""

def get_pages(soup):
    try:
        pages = soup.find(itemprop ="numberOfPages").get_text().split()[0]
        return int(pages)
    except AttributeError:
        return ""

def get_ratingValue(soup):
    try:
        ratingValue = soup.find(itemprop ="ratingValue").get_text().split()[0]
        return float(ratingValue)
    except AttributeError:
        return ""

def get_ratingCount(soup):
    try:
        test = []
        ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
        for x in ratingCount:
            test.append(x.get_text().split())
        return int(test[0][0].replace(",",""))
    except AttributeError:
        return ""

def get_reviewCount(soup):
    try:
        test = []
        ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
        for x in ratingCount:
            test.append(x.get_text().split())
        return int(test[1][0].replace(",",""))
    except AttributeError:
        return ""

def get_date(soup):
    try:
        regex = "by.*"
        date = soup.find_all(class_ = "row")
        date = re.sub(regex,"" ,str(date[1].contents[0]).replace("Published","")).strip()
        return date
    except AttributeError:
        return ""

def get_setting(soup):
    try:
        string = re.compile("/places/.")
        test = soup.find_all(href=string)

        places_list = " ".join([(str(a.contents[0])) for a in test])

        return places_list
    except AttributeError:
        return ""

def get_plot(soup):
    try:
        plot = soup.find(id="description")
        plot = plot.find_all(id= re.compile("freeText\.*"))

        final_plot = [] # It seems i need a list because joining on an empty string gives me a !"£$%&"@ blank

    # now there should not be a case where there are 3 span tags ...hopefully
        if len(plot) > 1:
            final_plot.append(plot[1].get_text())
        else:
            final_plot.append(plot[0].get_text())

        return final_plot[0]
    except AttributeError:
        return ""


def get_char(soup):
    try:
        string = re.compile("/characters/.")
        test = soup.find_all(href=string)

        char_list= " ".join([(str(a.contents[0])) for a in test])

        return char_list
    except AttributeError:
        return ""

def get_url(soup):
    try:
        regex = "https://www.goodreads.com/book/show/."
        ratingCount = soup.find(href =re.compile(regex), rel="canonical")
        return ratingCount.get('href')
    except AttributeError:
        return ""


In [14]:
# SECTION 2


eng_count = 9754
nb_html_articles = 100
page = 101
art = 1

for k in range(1,101):

    path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_"+str(page)
    os.chdir(path)
    
    
    for i in range(1,nb_html_articles+1):

        path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3\\page_"+str(page)
        os.chdir(path)
    
        html = open("article_"+str(art)+".html", "r", encoding="utf-8")
        soup = BeautifulSoup(html.read(), 'html.parser')
        try:
            if detect(get_plot(soup)) == "en":
                eng_count += 1
                art = art +1
                
       
                with open("article_"+str(eng_count)+".tsv", 'w', encoding="utf-8") as f_output:
                    test_list_tsv = [get_title(soup), 
                    get_series(soup), 
                    get_author(soup),
                    get_ratingValue(soup),
                    get_ratingCount(soup), 
                    get_reviewCount(soup),
                    get_plot(soup),
                    get_pages(soup),
                    get_date(soup),
                    get_setting(soup),
                    get_char(soup),
                    get_url(soup)]
            
                    header_list = ["bookTitle","bookSeries","bookAuthors","ratingValue","ratingCount","reviewCount","Plot","NumberofPages","Date","Characters","Setting","URL"]

                    path = "C:\\Users\\Eloisa\\Desktop\\UNI LORENZO\\ADM\\Homework 3"
                    os.chdir(path)

                    tsv_output = csv.writer(f_output, delimiter='\t')
                    tsv_output.writerow(header_list)
                    tsv_output.writerow(test_list_tsv)
                    
            else: 
                art = art +1
        except:
            language ="error"
        #ErrorCode.CantDetectError
    page = page + 1

FileNotFoundError: [Errno 2] No such file or directory: 'article_69.html'